# Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# # change working directory

# import os

# os.getcwd()
# os.chdir('/content/drive/MyDrive/AML_kaggle_challenge/Kaggle Competition')
# os.getcwd()

In [ ]:
# !pip install pykan

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
from pathlib import Path
from tqdm import tqdm
import torch.nn.functional as F

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pickle

import gc

import matplotlib.pyplot as plt

# from kan import KAN # kolmogorov arnold network

# Functions

In [ ]:
def load_data(path):
    data = dict(np.load(path, allow_pickle=True))
    return data

In [ ]:
# def contrastive_loss_with_multiple_positives(logits, label_indices):
#     """Remove model.temperature - use only learned scaling"""
#     # Create positive mask
#     positive_mask = (label_indices.unsqueeze(1) == label_indices.unsqueeze(0)).float()

#     # --- Text-to-Image Loss ---
#     # NO temperature scaling here - already applied before
#     logits_max_t2i, _ = torch.max(logits, dim=1, keepdim=True)
#     logits_stable_t2i = logits - logits_max_t2i.detach()

#     exp_sim_t2i = torch.exp(logits_stable_t2i)
#     positive_sim_t2i = torch.sum(exp_sim_t2i * positive_mask, dim=1)
#     all_sim_t2i = torch.sum(exp_sim_t2i, dim=1)

#     eps = 1e-9
#     loss_t2i = -torch.log(positive_sim_t2i / (all_sim_t2i + eps) + eps).mean()

#     # --- Image-to-Text Loss ---
#     logits_i2t = logits.T  # Just transpose, no scaling
#     logits_max_i2t, _ = torch.max(logits_i2t, dim=1, keepdim=True)
#     logits_stable_i2t = logits_i2t - logits_max_i2t.detach()

#     exp_sim_i2t = torch.exp(logits_stable_i2t)
#     positive_sim_i2t = torch.sum(exp_sim_i2t * positive_mask.T, dim=1)
#     all_sim_i2t = torch.sum(exp_sim_i2t, dim=1)

#     loss_i2t = -torch.log(positive_sim_i2t / (all_sim_i2t + eps) + eps).mean()

#     return (loss_t2i + loss_i2t) / 2.0

def contrastive_loss_with_multiple_positives(logits, label_indices, scale_penalty_weight=0.001):
    """With scale regularization to prevent explosion"""
    positive_mask = (label_indices.unsqueeze(1) == label_indices.unsqueeze(0)).float()

    # Remove self-comparisons
    positive_mask = positive_mask * (1 - torch.eye(positive_mask.size(0), device=positive_mask.device))

    # --- Text-to-Image Loss ---
    logits_max_t2i, _ = torch.max(logits, dim=1, keepdim=True)
    logits_stable_t2i = logits - logits_max_t2i.detach()

    exp_sim_t2i = torch.exp(logits_stable_t2i)
    positive_sim_t2i = torch.sum(exp_sim_t2i * positive_mask, dim=1)
    all_sim_t2i = torch.sum(exp_sim_t2i, dim=1)

    eps = 1e-9
    loss_t2i = -torch.log(positive_sim_t2i / (all_sim_t2i + eps) + eps).mean()

    # --- Image-to-Text Loss ---
    logits_i2t = logits.T
    logits_max_i2t, _ = torch.max(logits_i2t, dim=1, keepdim=True)
    logits_stable_i2t = logits_i2t - logits_max_i2t.detach()

    exp_sim_i2t = torch.exp(logits_stable_i2t)
    positive_sim_i2t = torch.sum(exp_sim_i2t * positive_mask.T, dim=1)
    all_sim_i2t = torch.sum(exp_sim_i2t, dim=1)

    loss_i2t = -torch.log(positive_sim_i2t / (all_sim_i2t + eps) + eps).mean()

    # Scale regularization (if scale is passed)
    scale_penalty = 0.0
    if hasattr(logits, 'scale_factor'):
        # Penalize large scale values
        scale_penalty = scale_penalty_weight * (logits.scale_factor ** 2)

    contrastive_loss = (loss_t2i + loss_i2t) / 2.0
    return contrastive_loss + scale_penalty




def calculate_mrr_text_to_image(model, data_loader, device):
    """Calculate Mean Reciprocal Rank (MRR) for text→image retrieval"""
    model.eval()
    reciprocal_ranks = []

    with torch.no_grad():
        for text_emb, image_emb, _ in tqdm(data_loader, desc="Calculating MRR "):
            text_emb, image_emb = text_emb.to(device), image_emb.to(device)

            pred_image_emb = model(text_emb)

            pred_image_emb = F.normalize(pred_image_emb, dim=1)

            sim_matrix = pred_image_emb @ image_emb.T  # (B, B)

            batch_size = len(text_emb)
            target_indices = torch.arange(batch_size).to(device)
            correct_scores = sim_matrix.diagonal()


            ranks = (sim_matrix > correct_scores.unsqueeze(1)).sum(dim=1) + 1

            reciprocal_ranks.extend((1.0 / ranks).cpu().tolist())
    mrr = sum(reciprocal_ranks) / len(reciprocal_ranks)
    return mrr


def train_model_combined(model, train_loader, val_loader, device, epochs, lr, model_path,
                        labels_train_indices, labels_val_indices):
    """Combined training with learnable temperature and MRR evaluation"""

    # Learnable temperature parameter (like in train_model_contrastive)
    initial_log_value = np.log(1 / 0.07)
    logit_scale = nn.Parameter(torch.empty(1).fill_(initial_log_value).to(device))

    # Optimizer for both model and temperature
    optimizer = optim.Adam([
        {'params': model.parameters()},
        {'params': [logit_scale], 'lr': 1e-2}  # Higher LR for temperature
    ], lr=lr, weight_decay=1e-5)

    # Scheduler
    # scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5)
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer,
        T_0=10,           # First restart after 10 epochs
        T_mult=1,         # Double the cycle length each time (1, 2, 4, 8...)
        eta_min=1e-6      # Minimum learning rate
    )

    current_lr = optimizer.param_groups[0]['lr']
    current_scale = logit_scale.exp().item()

    print(f"Initial learning rate: {current_lr:.2e}")
    print(f"Initial temperature: {current_scale:.4f}")

    best_val_mrr = -1.0
    best_val_loss = float('inf')
    Path(model_path).parent.mkdir(parents=True, exist_ok=True)

    for epoch in range(epochs):
        # Training phase
        model.train()
        train_loss = 0
        num_batches = 0

        for X_batch, y_batch, batch_label_indices in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            batch_label_indices = batch_label_indices.to(device)

            optimizer.zero_grad()

            # Forward pass
            pred_emb = model(X_batch)

            # Normalize embeddings
            pred_norm = F.normalize(pred_emb, p=2, dim=1)
            target_norm = F.normalize(y_batch, p=2, dim=1)

            # Compute similarity matrix
            logits = pred_norm @ target_norm.T

            # Apply learned temperature scaling
            with torch.no_grad():
                logit_scale.clamp_(0, 5)
                # logit_scale.clamp_(0, 4.605)  # max scale = 100
                # logit_scale.clamp_(0, 2.302)  # max scale = exp(2.302) ≈ 10
            scale_factor = logit_scale.exp()
            scaled_logits = logits * scale_factor

            # Multi-positive contrastive loss
            # loss = contrastive_loss_with_multiple_positives(
            #     scaled_logits, batch_label_indices, model
            # )

            loss = contrastive_loss_with_multiple_positives(scaled_logits, batch_label_indices)

            # Skip NaN losses
            if torch.isnan(loss):
                print("NaN loss detected! Skipping batch.")
                continue

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            train_loss += loss.item()
            num_batches += 1

        avg_train_loss = train_loss / num_batches if num_batches > 0 else float('inf')

        # Validation phase
        model.eval()
        val_loss = 0
        num_val_batches = 0

        with torch.no_grad():
            # Calculate validation loss
            for X_batch, y_batch, batch_label_indices in val_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                batch_label_indices = batch_label_indices.to(device)

                pred_emb = model(X_batch)
                pred_norm = F.normalize(pred_emb, p=2, dim=1)
                target_norm = F.normalize(y_batch, p=2, dim=1)
                logits = pred_norm @ target_norm.T

                # Apply same temperature scaling for validation
                scale_factor_val = logit_scale.exp()
                scaled_logits_val = logits * scale_factor_val

                # loss = contrastive_loss_with_multiple_positives(
                #     scaled_logits_val, batch_label_indices, model
                # )

                loss = contrastive_loss_with_multiple_positives(scaled_logits_val, batch_label_indices)

                if not torch.isnan(loss):
                    val_loss += loss.item()
                    num_val_batches += 1

        avg_val_loss = val_loss / num_val_batches if num_val_batches > 0 else float('inf')

        # Calculate MRR
        val_mrr = calculate_mrr_text_to_image(model, val_loader, device)

        # Get current learning rate and scale
        current_lr = optimizer.param_groups[0]['lr']
        current_scale = logit_scale.exp().item()

        print(f"Epoch {epoch+1}:")
        print(f"  Train Loss = {avg_train_loss:.6f}, Val Loss = {avg_val_loss:.6f}")
        print(f"  Val MRR = {val_mrr:.4f}, Scale = {current_scale:.4f}, LR = {current_lr:.2e}")
        # print(f"  Temperature = {model.temperature.item():.6f}")

        # Update scheduler based on MRR (we want to maximize MRR)
        scheduler.step(val_mrr)

        # Save best model based on MRR (primary) and loss (secondary)
        if val_mrr > best_val_mrr or (abs(val_mrr - best_val_mrr) < 1e-6 and avg_val_loss < best_val_loss):
            best_val_mrr = val_mrr
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), model_path)
            print(f"  ✓ Saved best model (MRR={val_mrr:.4f}, Loss={avg_val_loss:.6f})")

        print("-" * 60)

    return model

# Read Data

In [ ]:
# read data back
data = torch.load('/content/drive/MyDrive/AML kaggle competition/sandros_data_15_11_25.pt')

X_train = data['X_train']
X_train_scaled = data['X_train_scaled']
X_val = data['X_val']
X_val_scaled = data['X_val_scaled']

y_train = data['y_train']
y_train_scaled = data['y_train_scaled']
y_val = data['y_val']
y_val_scaled = data['y_val_scaled']

labels_train = data['labels_train']
labels_val = data['labels_val']

In [ ]:
import gc

del data

gc.collect()

27

# MLP

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim=1024, output_dim=1536, hidden_dim_1=1536, hidden_dim_2=2048, dropout_prob=0.5):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim_1),
            nn.BatchNorm1d(hidden_dim_1),
            # nn.LayerNorm(hidden_dim_1),
            nn.GELU(),
            nn.Dropout(dropout_prob),

            nn.Linear(hidden_dim_1, hidden_dim_2),
            nn.BatchNorm1d(hidden_dim_2),
            # nn.LayerNorm(hidden_dim_2),
            nn.GELU(),
            nn.Dropout(dropout_prob),

            nn.Linear(hidden_dim_2, output_dim),
        )

        # # Better temperature initialization
        # self.log_temperature = nn.Parameter(torch.log(torch.tensor(0.07)))

    def forward(self, x):
        return self.net(x)

In [ ]:
# class MLP(nn.Module):
#     def __init__(self, input_dim=1024, output_dim=1536, hidden_dim_1=1536, hidden_dim_2=2048, dropout_prob=0.5):
#         super().__init__()

#         self.net = nn.Sequential(
#             nn.Linear(input_dim, hidden_dim_1),
#             nn.LayerNorm(hidden_dim_1),
#             nn.GELU(),
#             nn.Dropout(dropout_prob),

#             nn.Linear(hidden_dim_1, hidden_dim_2),
#             nn.LayerNorm(hidden_dim_2),
#             nn.GELU(),
#             nn.Dropout(dropout_prob),

#             nn.Linear(hidden_dim_2, output_dim),
#         )

#         # Apply custom initialization
#         self._init_weights()

#     def _init_weights(self):
#         for module in self.modules():
#             if isinstance(module, nn.Linear):
#                 # Transformer-style initialization
#                 nn.init.xavier_uniform_(module.weight, gain=nn.init.calculate_gain('relu'))
#                 if module.bias is not None:
#                     nn.init.constant_(module.bias, 0.0)

#             elif isinstance(module, nn.LayerNorm):
#                 nn.init.constant_(module.weight, 1.0)
#                 nn.init.constant_(module.bias, 0.0)

#     def forward(self, x):
#         return self.net(x)

In [ ]:
# Initialize model
MODEL_PATH = "models/mlp.pth"
EPOCHS = 150
BATCH_SIZE = 2048
LR = 0.0005
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")



model = MLP().to(DEVICE)
print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")

# Create data loaders with proper shuffling
train_loader = DataLoader(
    TensorDataset(X_train_scaled, y_train, labels_train),
    batch_size=BATCH_SIZE,
    shuffle=True  # Important for contrastive learning
)
val_loader = DataLoader(
    TensorDataset(X_val_scaled, y_val, labels_val),
    batch_size=100,
    shuffle=False
)

# Train with combined approach
print("\n3. Training with combined approach...")
model = train_model_combined(
    model, train_loader, val_loader, DEVICE, EPOCHS, LR, MODEL_PATH,
    labels_train, labels_val
)

Parameters: 7,876,608

3. Training with combined approach...
Initial learning rate: 5.00e-04
Initial temperature: 14.2857


Calculating MRR : 100%|██████████| 125/125 [00:00<00:00, 466.45it/s]


Epoch 1:
  Train Loss = 19.764290, Val Loss = 20.648575
  Val MRR = 0.6056, Scale = 24.5270, LR = 5.00e-04
  ✓ Saved best model (MRR=0.6056, Loss=20.648575)
------------------------------------------------------------


Calculating MRR : 100%|██████████| 125/125 [00:00<00:00, 475.13it/s]


Epoch 2:
  Train Loss = 19.677386, Val Loss = 20.647210
  Val MRR = 0.7073, Scale = 38.9411, LR = 4.95e-04
  ✓ Saved best model (MRR=0.7073, Loss=20.647210)
------------------------------------------------------------


...

Epoch 148:
  Train Loss = 19.524236, Val Loss = 20.645102
  Val MRR = 0.9031, Scale = 118.3260, LR = 4.90e-04
------------------------------------------------------------


Calculating MRR : 100%|██████████| 125/125 [00:00<00:00, 419.90it/s]


Epoch 149:
  Train Loss = 19.502384, Val Loss = 20.645017
  Val MRR = 0.9046, Scale = 119.9859, LR = 4.90e-04
  ✓ Saved best model (MRR=0.9046, Loss=20.645017)
------------------------------------------------------------


Calculating MRR : 100%|██████████| 125/125 [00:00<00:00, 334.51it/s]

Epoch 150:
  Train Loss = 19.539334, Val Loss = 20.645118
  Val MRR = 0.9036, Scale = 121.0790, LR = 4.90e-04
------------------------------------------------------------


In [ ]:
# Load best model for evaluation
model = MLP().to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

# Evaluation

In [ ]:
def mrr(pred_indices: np.ndarray, gt_indices: np.ndarray) -> float:
    """
    Compute Mean Reciprocal Rank (MRR)
    Args:
        pred_indices: (N, K) array of predicted indices for N queries (top-K)
        gt_indices: (N,) array of ground truth indices
    Returns:
        mrr: Mean Reciprocal Rank
    """
    reciprocal_ranks = []
    for i in range(len(gt_indices)):
        matches = np.where(pred_indices[i] == gt_indices[i])[0]
        if matches.size > 0:
            reciprocal_ranks.append(1.0 / (matches[0] + 1))
        else:
            reciprocal_ranks.append(0.0)
    return np.mean(reciprocal_ranks)


def recall_at_k(pred_indices: np.ndarray, gt_indices: np.ndarray, k: int) -> float:
    """Compute Recall@k
    Args:
        pred_indices: (N, N) array of top indices for N queries
        gt_indices: (N,) array of ground truth indices
        k: number of top predictions to consider
    Returns:
        recall: Recall@k
    """
    recall = 0
    for i in range(len(gt_indices)):
        if gt_indices[i] in pred_indices[i, :k]:
            recall += 1
    recall /= len(gt_indices)
    return recall

import numpy as np

def ndcg(pred_indices: np.ndarray, gt_indices: np.ndarray, k: int = 100) -> float:
    """
    Compute Normalized Discounted Cumulative Gain (NDCG@k)
    Args:
        pred_indices: (N, K) array of predicted indices for N queries
        gt_indices: (N,) array of ground truth indices
        k: number of top predictions to consider
    Returns:
        ndcg: NDCG@k
    """
    ndcg_total = 0.0
    for i in range(len(gt_indices)):
        matches = np.where(pred_indices[i, :k] == gt_indices[i])[0]
        if matches.size > 0:
            rank = matches[0] + 1
            ndcg_total += 1.0 / np.log2(rank + 1)  # DCG (IDCG = 1)
    return ndcg_total / len(gt_indices)



@torch.inference_mode()
def evaluate_retrieval(translated_embd, image_embd, gt_indices, max_indices = 99, batch_size=100):
    """Evaluate retrieval performance using cosine similarity
    Args:
        translated_embd: (N_captions, D) translated caption embeddings
        image_embd: (N_images, D) image embeddings
        gt_indices: (N_captions,) ground truth image indices for each caption
        max_indices: number of top predictions to consider
    Returns:
        results: dict of evaluation metrics

    """
    # Compute similarity matrix
    if isinstance(translated_embd, np.ndarray):
        translated_embd = torch.from_numpy(translated_embd).float()
    if isinstance(image_embd, np.ndarray):
        image_embd = torch.from_numpy(image_embd).float()

    n_queries = translated_embd.shape[0]
    device = translated_embd.device

    # Prepare containers for the fragments to be reassembled
    all_sorted_indices = []
    l2_distances = []

    # Process in batches - the narrow gate approach
    for start_idx in range(0, n_queries, batch_size):
        batch_slice = slice(start_idx, min(start_idx + batch_size, n_queries))
        batch_translated = translated_embd[batch_slice]
        batch_img_embd = image_embd[batch_slice]

        # Compute similarity only for this batch
        batch_similarity = batch_translated @ batch_img_embd.T

        # Get top-k predictions for this batch
        batch_indices = batch_similarity.topk(k=max_indices, dim=1, sorted=True).indices.numpy()
        all_sorted_indices.append(gt_indices[batch_slice][batch_indices])

        # Compute L2 distance for this batch
        batch_gt = gt_indices[batch_slice]
        batch_gt_embeddings = image_embd[batch_gt]
        batch_l2 = (batch_translated - batch_gt_embeddings).norm(dim=1)
        l2_distances.append(batch_l2)

    # Reassemble the fragments
    sorted_indices = np.concatenate(all_sorted_indices, axis=0)

    # Apply the sacred metrics to the whole
    metrics = {
        'mrr': mrr,
        'ndcg': ndcg,
        'recall_at_1': lambda preds, gt: recall_at_k(preds, gt, 1),
        'recall_at_3': lambda preds, gt: recall_at_k(preds, gt, 3),
        'recall_at_5': lambda preds, gt: recall_at_k(preds, gt, 5),
        'recall_at_10': lambda preds, gt: recall_at_k(preds, gt, 10),
        'recall_at_50': lambda preds, gt: recall_at_k(preds, gt, 50),
    }

    results = {
        name: func(sorted_indices, gt_indices)
        for name, func in metrics.items()
    }

    l2_dist = torch.cat(l2_distances, dim=0).mean().item()
    results['l2_dist'] = l2_dist

    return results

In [ ]:
model.eval()
model.to(DEVICE)

with torch.no_grad():
    pred_embds_val = model(X_val_scaled.to(DEVICE)).cpu()
    pred_embds_train = model(X_train_scaled.to(DEVICE)).cpu()

print("Final evaluation on training set :")
print(evaluate_retrieval(pred_embds_train, y_train, labels_train))

print("Final evaluation on validation set :")
print(evaluate_retrieval(pred_embds_val, y_val, labels_val))

Final evaluation on training set :
{'mrr': np.float64(0.9494112997384394), 'ndcg': np.float64(0.9621445088073489), 'recall_at_1': 0.9117777777777778, 'recall_at_3': 0.9865688888888889, 'recall_at_5': 0.99408, 'recall_at_10': 0.9979822222222222, 'recall_at_50': 0.9998755555555555, 'l2_dist': 76.93505859375}
Final evaluation on validation set :
{'mrr': np.float64(0.9046314923389364), 'ndcg': np.float64(0.9281371693839972), 'recall_at_1': 0.84368, 'recall_at_3': 0.962, 'recall_at_5': 0.9804, 'recall_at_10': 0.9928, 'recall_at_50': 0.99984, 'l2_dist': 76.58287048339844}


# Submission

In [ ]:
# read scaler for features
with open('/content/drive/MyDrive/AML kaggle competition/scaler_x_15_11_25.pkl', 'rb') as f:
    sc_x = pickle.load(f)

/usr/local/lib/python3.12/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.7.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
test_data = load_data("/content/drive/MyDrive/AML kaggle competition/test.clean.npz")

test_embds = test_data['captions/embeddings']
test_embds = sc_x.transform(test_embds) # Scale the test caption embeddings
test_embds = torch.from_numpy(test_embds).float()

model.eval()
model.to(DEVICE)

with torch.no_grad():
    pred_embds = model(test_embds.to(DEVICE)).cpu()

In [ ]:
def generate_submission(sample_ids, translated_embeddings, output_file="submission.csv"):
    """
    Generate a submission.csv file from translated embeddings.
    """
    print("Generating submission file...")

    if isinstance(translated_embeddings, torch.Tensor):
        translated_embeddings = translated_embeddings.cpu().numpy()

    # Create a DataFrame with sample_id and embeddings

    df_submission = pd.DataFrame({'id': sample_ids, 'embedding': translated_embeddings.tolist()})

    df_submission.to_csv(output_file, index=False, float_format='%.17g')
    print(f"✓ Saved submission to {output_file}")

    return df_submission

In [ ]:
submission = generate_submission(test_data['captions/ids'], pred_embds,
                                 'submission_151125_v13.csv')

print(f"Model saved to: {MODEL_PATH}")

Generating submission file...
✓ Saved submission to submission_151125_v13.csv
Model saved to: models/mlp.pth
